In [1]:
from RRQ.faiss_interface import PQ as myPQ
from RRQ.faiss_interface import OPQ as myOPQ
import numpy as np
import RRQ.Tools as Tools
##########################
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.linalg import toeplitz

In [2]:
N_test = 10000
N_train = 10000
n = 512
numRep = 1


In [3]:
RhoList = [0,0.5,0.99]
#RhoList = [0.99]
MethodList = ['PQ']
mList = [256]
#pList = [1,2,4,5,10,20,25,50,100,200,250,500]
pList = [1,2,4,8,16,32,64,128,256]
for RhoInd,Rho in enumerate(RhoList):
    for Method in MethodList:
        for m in mList:
            exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_m' + str(m) + '_train = np.zeros((numRep,len(pList)+1))')
            exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_m' + str(m) + '_train[:,0] = 1')
            exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_m' + str(m) + '_test = np.zeros((numRep,len(pList)+1))')
            exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_m' + str(m) + '_test[:,0] = 1')
            #
            exec('rate_' + Method + '_Rho' + str(int(Rho*100)) + '_m' + str(m) + ' = np.zeros((numRep,len(pList)+1))')

In [4]:
for RepInd in range(numRep):
    print('.......................................  Repetition = ',RepInd + 1)
    for RhoInd,Rho in enumerate(RhoList):
        print('....................  Rho = ',Rho)
        F_train,F_test = Tools.data_generator(n,N_train,N_test,'AR1',SourceParam=Rho)
        norm_train = np.divide(np.linalg.norm(F_train) ** 2, np.prod(F_train.shape))
        norm_test = np.divide(np.linalg.norm(F_test) ** 2, np.prod(F_test.shape))
        for MethodInd,Method in enumerate(MethodList):
            print(' ############   ' + Method + '     A L G O R I T H M   ################')
            for m in mList:
                print(' ..... m = ',m, '  .........')
                for pInd,p in enumerate(pList):
                    print(' ..... p = ',p, '  .........')
                    if Method == 'PQ':
                        obj = myPQ(n,m,p)
                    elif Method == 'OPQ':
                        obj = myOPQ(n,m,p)
                    # training:
                    obj.train(F_train)
                    _,_ = obj.run(F_train)
                    exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_m' + str(m) + '_train[RepInd,pInd+1] = obj.distortion/norm_train')
                    exec('rate_' + Method + '_Rho' + str(int(Rho*100)) + '_m' + str(m) +       '[RepInd,pInd+1] = obj.rate')
                    # testing:
                    _,_ = obj.run(F_test)
                    exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_m' + str(m) + '_test[RepInd,pInd+1] = obj.distortion/norm_test')
       

.......................................  Repetition =  1
....................  Rho =  0
 ############   PQ     A L G O R I T H M   ################
 ..... m =  256   .........
 ..... p =  1   .........
 ..... p =  2   .........
 ..... p =  4   .........
 ..... p =  8   .........
 ..... p =  16   .........
 ..... p =  32   .........
 ..... p =  64   .........
 ..... p =  128   .........
 ..... p =  256   .........
....................  Rho =  0.5
 ############   PQ     A L G O R I T H M   ################
 ..... m =  256   .........
 ..... p =  1   .........
 ..... p =  2   .........
 ..... p =  4   .........
 ..... p =  8   .........
 ..... p =  16   .........
 ..... p =  32   .........
 ..... p =  64   .........
 ..... p =  128   .........
 ..... p =  256   .........
....................  Rho =  0.99
 ############   PQ     A L G O R I T H M   ################
 ..... m =  256   .........
 ..... p =  1   .........
 ..... p =  2   .........
 ..... p =  4   .........
 ..... p =  8   .....

In [5]:
# plt.plot(np.mean(rate_OPQ_Rho0_m256,axis=0).reshape(-1,1),np.mean(dist_OPQ_Rho0_m256_train,axis=0).reshape(-1,1))
# plt.plot(np.mean(rate_OPQ_Rho0_m256,axis=0).reshape(-1,1),np.mean(dist_OPQ_Rho0_m256_test,axis=0).reshape(-1,1))
# plt.grid()

In [6]:
curve_dict = {}
for RhoInd,Rho in enumerate(RhoList):
    for Method in MethodList:
        for m in mList:
            exec('x = np.mean(rate_' + Method + '_Rho' + str(int(Rho*100)) + '_m' + str(m) + ',axis=0)')
            exec('y = np.mean(dist_' + Method + '_Rho' + str(int(Rho*100)) + '_m' + str(m) + '_train,axis=0)')
            exec("curve_dict['Rho" + str(int(Rho*100))+ '_n' + str(n) + '_m' + str(m) +"_DR_"+ Method + "_train'] = (x,y)")
            exec('y = np.mean(dist_' + Method + '_Rho' + str(int(Rho*100)) + '_m' + str(m) + '_test,axis=0)')
            exec("curve_dict['Rho" + str(int(Rho*100))+ '_n' + str(n)+ '_m' + str(m) +"_DR_"+ Method + "_test'] = (x,y)") 

In [7]:
# Saving the results for PGFplott:
PGF_path = 'PGF/dat/'
ExpName = 'SyntheticAR1_VQ'
#
for curve_name, curve_xy in curve_dict.items():
    fname = ExpName + '_' + curve_name + '.dat'
    print(fname)
    x = curve_xy[0].astype(float).reshape(-1)
    y = curve_xy[1].astype(float).reshape(-1)
    np.savetxt(PGF_path + fname, np.transpose([x,y]), fmt='%8f', delimiter='   ')

SyntheticAR1_VQ_Rho0_n512_m256_DR_PQ_train.dat
SyntheticAR1_VQ_Rho0_n512_m256_DR_PQ_test.dat
SyntheticAR1_VQ_Rho50_n512_m256_DR_PQ_train.dat
SyntheticAR1_VQ_Rho50_n512_m256_DR_PQ_test.dat
SyntheticAR1_VQ_Rho99_n512_m256_DR_PQ_train.dat
SyntheticAR1_VQ_Rho99_n512_m256_DR_PQ_test.dat
